In [ ]:
import os
from scipy import misc
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import tensorflow

In [ ]:
def create_dir(directory):
    os.makedirs(directory)

In [ ]:
def check_dir(directory):
    if not os.path.exists(directory):
        create_dir(directory)

In [ ]:
width = height = 64
image_size = (width, height)
output = '../output'+str(width)

### Split and transform train stratify data to train and val

In [ ]:
%%time
dir_train_transformed = os.path.join(output, 'train')
dir_val_transformed = os.path.join(output, 'validation')

dir_train = '../input/train'
test_size = 0.25
    
for ind, folder in enumerate(os.listdir(dir_train)):
        subfolder = os.path.join(dir_train, folder)
        if os.path.isdir(subfolder):
            image_class = folder[:3]
            
            val_size = int(test_size * len(os.listdir(subfolder)))
            val_count = 0
            dir_train_to_save = os.path.join(dir_train_transformed, image_class)
            dir_val_to_save = os.path.join(dir_val_transformed, image_class)
            check_dir(dir_train_to_save)
            check_dir(dir_val_to_save)
            
            if ind % 10 == 0:
                print('folder - {}, validation size - {}, total size - {}'.format(folder, val_size, len(os.listdir(subfolder))))
            for image_file in os.listdir(subfolder):
                if not image_file.startswith('.'):
                    name = os.path.join(subfolder, image_file)
                    image = misc.imread(name)
                    if len(image.shape) == 2:
                        image = np.stack((image,)*3)
                    image = misc.imresize(image, image_size, interp='bicubic')
                    if val_count < val_size:
                        fold_to_save = os.path.join(dir_val_to_save, image_file)
                        misc.imsave(fold_to_save, image)
                        val_count = val_count + 1
                    else:
                        fold_to_save = os.path.join(dir_train_to_save, image_file)
                        misc.imsave(fold_to_save, image)

### Split to balanced classes

In [ ]:
%%time
dir_train_transformed = os.path.join(output, 'train')
dir_val_transformed = os.path.join(output, 'validation')

dir_train = '../input/train'
    
for ind, folder in enumerate(os.listdir(dir_train)):
        subfolder = os.path.join(dir_train, folder)
        if os.path.isdir(subfolder):
            image_class = folder[:3]
            val_count = 0
            dir_train_to_save = os.path.join(dir_train_transformed, image_class)
            dir_val_to_save = os.path.join(dir_val_transformed, image_class)
            check_dir(dir_train_to_save)
            check_dir(dir_val_to_save)
            
            if ind % 10 == 0:
                print('folder - {}'.format(folder))
            for image_file in os.listdir(subfolder):
                if not image_file.startswith('.'):
                    name = os.path.join(subfolder, image_file)
                    image = misc.imread(name)
                    if len(image.shape) == 2:
                        image = np.stack((image,)*3)
                    image = misc.imresize(image, image_size, interp='bicubic')
                    if val_count % 5 == 0 and val_count < 50:
                        fold_to_save = os.path.join(dir_val_to_save, image_file)
                        misc.imsave(fold_to_save, image)
                        val_count = val_count + 1
                    else:
                        fold_to_save = os.path.join(dir_train_to_save, image_file)
                        misc.imsave(fold_to_save, image)
                        val_count = val_count + 1

### Transform test data

In [ ]:
%%time
dir_test_transformed = os.path.join(output, 'test/test')
dir_test = '../input/test'

check_dir(dir_test_transformed)
for ind, image_name in enumerate(os.listdir(dir_test)):
        if ind % 600 == 0:
            print('{}'.format(ind))
        if not image_name.startswith('.'):
            name = os.path.join(dir_test, image_name)
            image = misc.imread(name)
            if len(image.shape) == 2:
                image = np.stack((image,)*3)
            image = misc.imresize(image, image_size, interp='bicubic')
            fold_to_save = os.path.join(dir_test_transformed, image_name)
            misc.imsave(fold_to_save, image)

In [ ]:
def save(dir_from, dir_to):
    for ind, folder in enumerate(os.listdir(dir_from)):
        subfolder = os.path.join(dir_from, folder)
        if os.path.isdir(subfolder):
            image_class = folder[:3]
            dir_train_to_save = os.path.join(dir_to, image_class)
            check_dir(dir_train_to_save)
            
            if ind % 10 == 0:
                print('folder - {}'.format(folder))
            for image_file in os.listdir(subfolder):
                if not image_file.startswith('.'):
                    name = os.path.join(subfolder, image_file)
                    image = misc.imread(name)
                    if len(image.shape) == 2:
                        image = np.stack((image,)*3)
                    image = misc.imresize(image, image_size, interp='bicubic')
                    fold_to_save = os.path.join(dir_train_to_save, image_file)
                    misc.imsave(fold_to_save, image)

In [ ]:
save('data/train', 'data/train_resized')

In [ ]:
dir_train = 'data/train_resized'
dir_train_augmented = 'data/train_augmented'
samples_threshhold = 775

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

for ind, folder in enumerate(os.listdir(dir_train)):
    subfolder = os.path.join(dir_train, folder)
    if os.path.isdir(subfolder):            
        dir_train_to_save = os.path.join(dir_train_transformed, image_class)
        check_dir(dir_train_to_save)

        if ind % 20 == 0:
            print('folder - {}'.format(folder))
        needed_augmented_images = samples_threshhold - len(os.listdir(subfolder))
        dir_to_save = os.path.join(dir_train_augmented, folder)
        check_dir(dir_to_save)
        
        print(needed_augmented_images, ' ###### ', folder)
        for image_file in os.listdir(subfolder):
            if not image_file.startswith('.'):
                name = os.path.join(subfolder, image_file)
                image = misc.imread(name)
                #misc.imsave(dir_to_save, image)
                
                image = image.reshape((1,) + image.shape)
                
                for batch in datagen.flow(image, batch_size=1,
                      save_to_dir=dir_to_save, save_format='jpeg'):
                    needed_augmented_images -= 1
                    if needed_augmented_images < 0:
                        break

### Transform images for keras

In [ ]:
train_df = pd.read_csv('data/data_analys.csv', index_col = 0)

In [ ]:
images = np.zeros((train_df.shape[0], 64, 64, 3), dtype='uint8')
targets = np.zeros((train_df.shape[0],), dtype='object')

In [ ]:
dir_train = 'data/train_transformed'
img_size=(64, 64)

i =0
for ind, image_name in enumerate(os.listdir(dir_train)):
    image_file = os.path.join(dir_train, image_name)
    if ind % 2000 == 0:
        print('{}'.format(ind))
    if not image_file.startswith('.'):
        #name = os.path.join(subfolder, image_file)
        image = misc.imread(image_file)
        if len(image.shape) == 2:
            image = np.stack((image,)*3)
        images[i] = misc.imresize(image, img_size, interp='bicubic')
        targets[i] = int(image_name[:3])
        i = i+1

In [ ]:
np.save('data/keras/train_images.npy', images)
np.save('data/keras/targets.npy', targets)

In [ ]:
%%time
dir_test_name = 'data/test_transformed'
dir_test = os.listdir(dir_test_name)
img_size=(64, 64)

test_images = np.zeros((len(dir_test), img_size[0], img_size[1], 3), dtype='uint8')

i = 0
for ind, image_name in enumerate(os.listdir(dir_test_name)):
        if ind % 600 == 0:
            print('{}'.format(ind))
        if not image_name.startswith('.'):
            name = os.path.join(dir_test_name, image_name)
            image = misc.imread(name)
            if len(image.shape) == 2:
                image = np.stack((image,)*3)
            test_images[i] = misc.imresize(image, img_size, interp='bicubic')
            i = i+1

In [ ]:
np.save('data/keras/test_images.npy', test_images)